In [1]:
from urllib.request import urlopen

from bs4 import BeautifulSoup
from statistics import mean 
import pandas as pd
import re
from datetime import datetime,timedelta 
import requests
import warnings
warnings.filterwarnings("ignore")

In [2]:
#даты
date_now=datetime.today().date()
day_now=date_now.day #<--- день сегодня
date_tomorrow=date_now + timedelta(days=1)
day_tomorrow=(date_now + timedelta(days=1)).day #<--- день завтра
day_after_tomorrow=(date_now + timedelta(days=2)).day #<--- день послезавтра

In [3]:
#считываем страницу с прогнозом Яндекса через read_html, получаем необработанные данные 
ya_df=pd.read_html('https://yandex.ru/pogoda/details?lat=55.753215&lon=37.622504&via=ms')

#каждый день находится в отдельной ячейке, прилепим их их друг под другом
ya_new=pd.concat([ya_df[0],ya_df[1],ya_df[2],ya_df[3],ya_df[4],ya_df[5],ya_df[6],ya_df[7],ya_df[8],ya_df[9]],ignore_index=True)
ya_new.columns=['temp','nan','weather','press','wet','wind','t_eff']

## ЯНДЕКС

In [4]:
#поработаем со столбцом температуры при помощи регулярных выражений
temp_clean=[]
for i in range(0,ya_new.shape[0]):
    t_str=ya_new['temp'][i]
    temp_str=re.search(r'[+ -]\d{1,2}\…[+ -]\d{1,2}',t_str)
    temp_clean.append(temp_str.group(0))
    
temp_fin=pd.Series(temp_clean) 
temp_fin=temp_fin.str.rsplit('…',expand=True)

temp_fin.columns=['t_down','t_up']   

In [5]:
#поработаем со столбцом ветра

wind_clean=[]
for i in range(0,ya_new.shape[0]):
    w_str=ya_new['wind'][i]
    wind_str=re.search(r'\d{1,2},\d',w_str)
    wind_clean.append(wind_str.group(0))
    
wind_fin=pd.Series(wind_clean)  
wind_fin=wind_fin.str.replace(',','.')

In [6]:
#поработаем со столбцом эффективной температуры
t_eff=[]
for i in range(0,ya_new.shape[0]):
    string=ya_new['t_eff'][i]
    if string[0]=='−':
        s_1=re.sub('[^0-9]','', string)
        #end=(len(s_1)/2)
        s_fin='−'+s_1[:len(s_1)//2]
    if string[0] != '−':  
        s_1=re.sub('[^0-9]','', string)
        s_fin=s_1[:len(s_1)//2]
    t_eff.append(s_fin)
    
temp_eff=pd.Series(t_eff) 

In [7]:
#собираем в табличку
list_days=[10*['утро','день','вечер','ночь']]
list_days=pd.DataFrame(list_days).T

ya_full=pd.concat([list_days,temp_fin,ya_new[['press','wet']],wind_fin,temp_eff,ya_new['weather']],axis=1)

ya_full.columns=['time','t_down','t_up','press','wet','wind','t_eff','weather']

#убираем "грязь"
ya_full['wet']=ya_full['wet'].str.replace('%','')
ya_full['t_down']=ya_full['t_down'].str.replace('−','-')
ya_full['t_up']=ya_full['t_up'].str.replace('−','-')
ya_full['t_eff']=ya_full['t_eff'].str.replace('−','-')


In [8]:
#поработаем с форматами
ya_full[['t_down','t_up','wet']]=ya_full[['t_down','t_up','wet']].astype('int')
ya_full['wind']=ya_full['wind'].astype('float')
ya_full['t_eff']=ya_full['t_eff'].astype('int')

#считаем среднюю температуру
ya_full['t_mean']=(ya_full['t_up']+ya_full['t_down'])/2

ya_full.head(20)

,time,t_down,t_up,press,wet,wind,t_eff,weather,t_mean
0,утро,6,13,755,55,2.3,7,Ясно,9.5
1,день,14,15,754,31,2.3,12,Облачно с прояснениями,14.5
2,вечер,12,15,752,43,2.0,11,Пасмурно,13.5
3,ночь,7,10,752,66,2.1,5,Облачно с прояснениями,8.5
4,утро,7,12,752,56,3.0,7,Облачно с прояснениями,9.5
5,день,11,13,751,45,3.5,8,Небольшой дождь,12.0
6,вечер,5,9,751,68,4.2,2,Небольшой дождь,7.0
7,ночь,1,5,750,83,4.6,-2,Пасмурно,3.0
8,утро,1,4,752,78,6.5,-4,Пасмурно,2.5
9,день,4,5,752,68,6.9,-3,Пасмурно,4.5
